In [21]:
import pandas as pd
from mlxtend.frequent_patterns import association_rules, apriori

# Read Dataset

In [22]:
sales_receipts = pd.read_csv('dataset/201904 sales reciepts.csv')
sales_receipts.head()

,transaction_id,transaction_date,transaction_time,sales_outlet_id,staff_id,customer_id,instore_yn,order,line_item_id,product_id,quantity,line_item_amount,unit_price,promo_item_yn
0,7,2019-04-01,12:04:43,3,12,558,N,1,1,52,1,2.50,2.50,N
1,11,2019-04-01,15:54:39,3,17,781,N,1,1,27,2,7.00,3.50,N
2,19,2019-04-01,14:34:59,3,17,788,Y,1,1,46,2,5.00,2.50,N
3,32,2019-04-01,16:06:04,3,12,683,N,1,1,23,2,5.00,2.50,N
4,33,2019-04-01,19:18:37,3,17,99,Y,1,1,34,1,2.45,2.45,N


In [23]:
product = pd.read_csv('dataset/product_2.csv')
product.head()

,product_id,product_group,product_category,product_type,product,product_description,unit_of_measure,current_wholesale_price,current_retail_price,tax_exempt_yn,promo_yn,new_product_yn
0,1,Soccer Jerseys,Men's Home Jersey,Classic Jersey,Barcelona Home Classic Jersey,Experience Barcelona's heritage with this clas...,Standard,14.40,$18.00,Y,N,N
1,2,Soccer Jerseys,Men's Home Jersey,Retro Jersey,Barcelona Home Retro Jersey,"Inspired by classic designs, this retro home j...",Standard,14.40,$18.00,Y,N,N
2,3,Soccer Jerseys,Men's Home Jersey,Performance Jersey,Barcelona Home Performance Jersey,"Engineered for performance, this home jersey f...",Standard,11.80,$14.75,Y,N,N
3,4,Soccer Jerseys,Men's Home Jersey,Premium Home Jersey,Barcelona Home Premium Jersey,Experience premium quality with hand-finished ...,Standard,16.36,$20.45,Y,N,N
4,5,Soccer Jerseys,Men's Home Jersey,Lightweight Jersey,Barcelona Home Lightweight Jersey,Stay cool and agile on the field with this lig...,Standard,12.00,$15.00,Y,N,N


# Data Wrangling

## Merge data

In [24]:
sales_receipts = sales_receipts[["transaction_id", "transaction_date", "sales_outlet_id", "customer_id", "product_id", "quantity", ]]
product = product[["product_id", "product_category", "product"]]

dataset = pd.merge(sales_receipts, product, how="left", on="product_id")
dataset

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product
0,7,2019-04-01,3,558,52,1,Crew Socks,Barcelona Performance Socks - Regular
1,11,2019-04-01,3,781,27,2,Men's Cleats,Barcelona Men's Eco Cleats - Large
2,19,2019-04-01,3,788,46,2,Crew Socks,Barcelona Crew Socks - Comfort Fit
3,32,2019-04-01,3,683,23,2,Men's Cleats,Barcelona Men's Speed Cleats - Regular
4,33,2019-04-01,3,99,34,1,Women's Cleats,Barcelona Women's Precision Cleats - Small
...,...,...,...,...,...,...,...,...
49889,753,2019-04-29,8,0,30,1,Men's Cleats,Barcelona Men's Pro Cleats - Large
49890,756,2019-04-29,8,8412,25,2,Men's Cleats,Barcelona Men's Eco Cleats - Small
49891,759,2019-04-29,8,0,31,1,Men's Cleats,Barcelona Men's Elite Cleats - Small
49892,763,2019-04-29,8,8030,44,2,Crew Socks,Barcelona Crew Socks - Regular Breathable


## Remove sizes

In [25]:
dataset[dataset["product"].str.contains("Socks")]['product'].unique()

array(['Barcelona Performance Socks - Regular',
       'Barcelona Crew Socks - Comfort Fit',
       'Barcelona Crew Socks - Classic Large',
       'Barcelona Crew Socks - Lightweight Large',
       'Barcelona Crew Socks - Comfort Fit Large',
       'Barcelona Crew Socks - Classic',
       'Barcelona Performance Socks - Large',
       'Barcelona Crew Socks - Lightweight',
       'Barcelona Elite Socks - Large', 'Barcelona Elite Socks - Regular',
       'Barcelona Crew Socks - Large Breathable',
       'Barcelona Crew Socks - Large', 'Barcelona Crew Socks - Regular',
       'Barcelona Dynamic Socks - Regular',
       'Barcelona Dynamic Socks - Large',
       'Barcelona Crew Socks - Regular Breathable'], dtype=object)

In [26]:
dataset["product"].nunique()

80

In [27]:
dataset["product"] = dataset["product"].str.replace(' - Regular', '')
dataset["product"] = dataset["product"].str.replace(' - Comfort Fit', '')
dataset["product"] = dataset["product"].str.replace(' - Classic Large', '')
dataset["product"] = dataset["product"].str.replace(' - Lightweight Large', '')
dataset["product"] = dataset["product"].str.replace(' - Comfort Fit Large', '')
dataset["product"] = dataset["product"].str.replace(' - Classic', '')
dataset["product"] = dataset["product"].str.replace(' - Large', '')
dataset["product"] = dataset["product"].str.replace(' - Lightweight', '')
dataset["product"] = dataset["product"].str.replace(' - Regular Breathable', '')
dataset["product"] = dataset["product"].str.replace(' - Small', '')
dataset["product"] = dataset["product"].str.replace(' - Special', '')
dataset["product"] = dataset["product"].str.replace(' - Large Breathable', '')

""" dataset['product']= dataset['product'].str.replace(' Rg','')
dataset['product']= dataset['product'].str.replace(' Sm','')
dataset['product']= dataset['product'].str.replace(' Lg','') """

In [28]:
dataset["product"].nunique()

56

In [29]:
print(dataset["product"].unique())

['Barcelona Performance Socks' "Barcelona Men's Eco Cleats"
 'Barcelona Crew Socks' "Barcelona Men's Speed Cleats"
 "Barcelona Women's Precision Cleats" "Barcelona Women's Speed Cleats"
 'Barcelona Eco Keeper Gloves' 'Barcelona Crew Socks Large'
 "Barcelona Women's Advanced Cleats" "Barcelona Women's Classic Cleats"
 'Barcelona Keeper Gloves' "Barcelona Men's Pro Cleats"
 'Barcelona Signature Match Shorts' 'Barcelona Elite Socks'
 'Barcelona Crew Socks Breathable'
 'Barcelona Limited Edition Match Shorts' "Barcelona Men's Elite Cleats"
 'Barcelona Elite Match Shorts' 'Barcelona Dynamic Socks'
 'Barcelona Utility Shorts' 'Barcelona Performance Match Shorts'
 'Barcelona Sport Shorts' 'Barcelona Retro Match Shorts'
 'Barcelona Anniversary Match Shorts' 'Barcelona Classic Match Shorts'
 'Barcelona Casual Shorts' 'Barcelona Training Shorts'
 'Barcelona Training Cone Set' 'Barcelona Away Classic Jersey'
 'Barcelona Team Headband' 'Barcelona Away Modern Jersey'
 'Barcelona Home Signature Jers

In [ ]:
""" # products_to_take = ['Cappuccino', 'Latte', 'Espresso shot',  \
                     'Dark chocolate','Sugar Free Vanilla syrup', 'Chocolate syrup',\
                    'Carmel syrup', 'Hazelnut syrup', 'Ginger Scone',  \
                    'Chocolate Croissant', 'Jumbo Savory Scone', 'Cranberry Scone', 'Hazelnut Biscotti',\
                    'Croissant', 'Almond Croissant', 'Oatmeal Scone', 'Chocolate Chip Biscotti',\
                    'Ginger Biscotti',\
                   ] """
                   
        

In [30]:
# dataset = dataset[dataset['product'].isin(products_to_take)]

In [31]:
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product
0,7,2019-04-01,3,558,52,1,Crew Socks,Barcelona Performance Socks
1,11,2019-04-01,3,781,27,2,Men's Cleats,Barcelona Men's Eco Cleats
2,19,2019-04-01,3,788,46,2,Crew Socks,Barcelona Crew Socks
3,32,2019-04-01,3,683,23,2,Men's Cleats,Barcelona Men's Speed Cleats
4,33,2019-04-01,3,99,34,1,Women's Cleats,Barcelona Women's Precision Cleats


In [32]:
dataset[['product', 'product_category']].drop_duplicates().reset_index(drop=True)

,product,product_category
0,Barcelona Performance Socks,Crew Socks
1,Barcelona Men's Eco Cleats,Men's Cleats
2,Barcelona Crew Socks,Crew Socks
3,Barcelona Men's Speed Cleats,Men's Cleats
4,Barcelona Women's Precision Cleats,Women's Cleats
5,Barcelona Women's Speed Cleats,Women's Cleats
6,Barcelona Eco Keeper Gloves,Goalkeeper Gloves
7,Barcelona Crew Socks Large,Crew Socks
8,Barcelona Women's Advanced Cleats,Women's Cleats
9,Barcelona Women's Classic Cleats,Women's Cleats


# Clean Transactions

In [34]:
dataset['transaction'] = dataset['transaction_id'].astype(str) + "_"+dataset['customer_id'].astype(str)

In [35]:
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product,transaction
0,7,2019-04-01,3,558,52,1,Crew Socks,Barcelona Performance Socks,7_558
1,11,2019-04-01,3,781,27,2,Men's Cleats,Barcelona Men's Eco Cleats,11_781
2,19,2019-04-01,3,788,46,2,Crew Socks,Barcelona Crew Socks,19_788
3,32,2019-04-01,3,683,23,2,Men's Cleats,Barcelona Men's Speed Cleats,32_683
4,33,2019-04-01,3,99,34,1,Women's Cleats,Barcelona Women's Precision Cleats,33_99


In [36]:
num_of_items_for_each_transaction = dataset['transaction'].value_counts().reset_index()
num_of_items_for_each_transaction.head()

,transaction,count
0,301_0,42
1,323_0,40
2,151_0,40
3,9_0,39
4,75_0,39


In [38]:
num_of_items_for_each_transaction[num_of_items_for_each_transaction['count']==1]

,transaction,count
7168,360_8077,1
7169,3979_0,1
7170,375_8169,1
7171,4087_317,1
7172,629_5783,1
...,...,...
23276,1470_8208,1
23277,1466_8186,1
23278,1462_8124,1
23279,1459_8250,1


In [39]:
valid_transactions = num_of_items_for_each_transaction[(num_of_items_for_each_transaction['count']>1)]['transaction'].tolist()

valid_transactions[:10]

['301_0',
 '323_0',
 '151_0',
 '9_0',
 '75_0',
 '76_0',
 '213_0',
 '51_0',
 '266_0',
 '137_0']

In [40]:
dataset = dataset[dataset['transaction'].isin(valid_transactions)]

In [42]:
dataset.shape

(33781, 9)

# Product Trends

In [43]:
dataset["product_category"].value_counts()

product_category
Crew Socks                8621
Shorts                    7406
Women's Cleats            6038
Men's Cleats              5399
Accessories               2413
Goalkeeper Gloves         2193
Training Equipment         531
Men's Home Jersey          454
Men's Away Jersey          303
Women's Jersey             196
Limited Edition Jersey     148
Clothing                    79
Name: count, dtype: int64

In [44]:
dataset["product"].value_counts()

product
Barcelona Crew Socks                      3760
Barcelona Women's Classic Cleats          1988
Barcelona Men's Pro Cleats                1640
Barcelona Men's Eco Cleats                1610
Barcelona Men's Speed Cleats              1593
Barcelona Women's Precision Cleats        1591
Barcelona Women's Advanced Cleats         1361
Barcelona Crew Socks Breathable           1119
Barcelona Eco Keeper Gloves               1114
Barcelona Elite Socks                     1103
Barcelona Women's Speed Cleats            1098
Barcelona Performance Socks               1084
Barcelona Keeper Gloves                   1079
Barcelona Dynamic Socks                   1042
Barcelona Performance Match Shorts         945
Barcelona Training Shorts                  785
Barcelona Limited Edition Match Shorts     659
Barcelona Retro Match Shorts               657
Barcelona Sport Shorts                     655
Barcelona Classic Match Shorts             633
Barcelona Anniversary Match Shorts         631
Barce

# Popularity Recommendation Engine

In [45]:
product_recommendation = dataset.groupby(["product", "product_category"]).count().reset_index()

In [48]:
product_recommendation = product_recommendation[["product", "product_category", "transaction_id"]]
product_recommendation = product_recommendation.rename(columns = {"transaction_id": "number_of_transactions"})

In [49]:
product_recommendation.head()

,product,product_category,number_of_transactions
0,Barcelona Anniversary Edition Jersey,Limited Edition Jersey,50
1,Barcelona Anniversary Match Shorts,Shorts,631
2,Barcelona Away Aero Jersey,Men's Away Jersey,44
3,Barcelona Away Classic Jersey,Men's Away Jersey,64
4,Barcelona Away Custom Jersey,Men's Away Jersey,47


In [51]:
product_recommendation.to_csv('api/recommendation_objects/popularity_recommendation.csv',index=False)

# Apriori Recommendation Engine

In [57]:
train_basket = (dataset.groupby(['transaction', 'product'])['product'].count().reset_index(name="Count"))
train_basket.head()

,transaction,product,Count
0,1000_0,Barcelona Crew Socks,1
1,1000_0,Barcelona Elite Socks,1
2,1000_0,Barcelona Keeper Gloves,1
3,1000_0,Barcelona Men's Pro Cleats,1
4,1000_0,Barcelona Men's Speed Cleats,1


In [54]:
my_basket = train_basket.pivot_table(index='transaction', columns='product', values='Count').fillna(0)
my_basket.head(10)

product,Barcelona Anniversary Edition Jersey,Barcelona Anniversary Match Shorts,Barcelona Away Aero Jersey,Barcelona Away Classic Jersey,Barcelona Away Custom Jersey,Barcelona Away Heritage Jersey,Barcelona Away Modern Jersey,Barcelona Away Sleek Jersey,Barcelona Casual Shorts,Barcelona Classic Match Shorts,...,Barcelona Training Shorts,Barcelona Utility Shorts,Barcelona Women's Advanced Cleats,Barcelona Women's Classic Cleats,Barcelona Women's Classic Jersey,Barcelona Women's Limited Jersey,Barcelona Women's Performance Jersey,Barcelona Women's Precision Cleats,Barcelona Women's Retro Jersey,Barcelona Women's Speed Cleats
transaction,,,,,,,,,,,,,,,,,,,,,
1000_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001_8306,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1002_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1002_5313,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1003_5559,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003_8031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1004_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
def encode_units(x):
    if x<= 0:
        return 0
    if x >0:
        return 1

my_basket_sets = my_basket.applymap(encode_units)
my_basket_sets.head(10)

/var/folders/20/fx1vp06158bgqhgm1r3vsjhw0000gn/T/ipykernel_57670/3402650987.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  my_basket_sets = my_basket.applymap(encode_units)


product,Barcelona Anniversary Edition Jersey,Barcelona Anniversary Match Shorts,Barcelona Away Aero Jersey,Barcelona Away Classic Jersey,Barcelona Away Custom Jersey,Barcelona Away Heritage Jersey,Barcelona Away Modern Jersey,Barcelona Away Sleek Jersey,Barcelona Casual Shorts,Barcelona Classic Match Shorts,...,Barcelona Training Shorts,Barcelona Utility Shorts,Barcelona Women's Advanced Cleats,Barcelona Women's Classic Cleats,Barcelona Women's Classic Jersey,Barcelona Women's Limited Jersey,Barcelona Women's Performance Jersey,Barcelona Women's Precision Cleats,Barcelona Women's Retro Jersey,Barcelona Women's Speed Cleats
transaction,,,,,,,,,,,,,,,,,,,,,
1000_0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1001_0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1001_8306,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1002_0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
1002_5313,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1003_0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
1003_5559,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1003_8031,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1004_0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
# Uses Apriori algorithm to find frequent itemsets (items that appear in atleast 5% of all transactions)
# Support values = probability of appearing in a transaction
frequent_items = apriori(my_basket_sets,min_support=0.05,use_colnames=True)
frequent_items.head(20)

/opt/homebrew/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.082310,(Barcelona Anniversary Match Shorts)
1,0.082031,(Barcelona Casual Shorts)
2,0.080915,(Barcelona Classic Match Shorts)
3,0.299665,(Barcelona Crew Socks)
4,0.120954,(Barcelona Crew Socks Breathable)
5,0.063058,(Barcelona Crew Socks Large)
6,0.116211,(Barcelona Dynamic Socks)
7,0.122907,(Barcelona Eco Keeper Gloves)
8,0.078265,(Barcelona Elite Match Shorts)
9,0.119420,(Barcelona Elite Socks)


In [62]:
# Rules are filtered based on lift, which measures how strongly the presence of one item (antecedent) increases the likelihood of another (consequent)
# only rules where lift >= 1 (association is better than random choice)
# antecedents = items that appear first in rule
# consequents = items that are recommended when antecedents appear.
# support = how often both items appear together
# confidence = probability if customer buys antecedent they'll also buy consequent
# lift = how much more likely the consequent is bought together when antecedent is present
rules_basket = association_rules(frequent_items,metric="lift",min_threshold=1)
rules_basket.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(Barcelona Crew Socks Breathable),(Barcelona Crew Socks),0.120954,0.299665,0.063756,0.527105,1.758980,1.0,0.027510,1.480952,0.490860,0.178655,0.324759,0.369931
1,(Barcelona Crew Socks),(Barcelona Crew Socks Breathable),0.299665,0.120954,0.063756,0.212756,1.758980,1.0,0.027510,1.116612,0.616118,0.178655,0.104433,0.369931
2,(Barcelona Crew Socks),(Barcelona Dynamic Socks),0.299665,0.116211,0.059989,0.200186,1.722611,1.0,0.025164,1.104994,0.598979,0.168561,0.095017,0.358196
3,(Barcelona Dynamic Socks),(Barcelona Crew Socks),0.116211,0.299665,0.059989,0.516206,1.722611,1.0,0.025164,1.447590,0.474645,0.168561,0.309197,0.358196
4,(Barcelona Crew Socks),(Barcelona Eco Keeper Gloves),0.299665,0.122907,0.061523,0.205307,1.670423,1.0,0.024692,1.103688,0.573082,0.170402,0.093947,0.352937


In [64]:
rules_basket[rules_basket['antecedents']=={"Barcelona Crew Socks"}].sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
20,(Barcelona Crew Socks),(Barcelona Women's Classic Cleats),0.299665,0.211775,0.083984,0.280261,1.323392,1.0,0.020523,1.095154,0.348927,0.196475,0.086887,0.338418
13,(Barcelona Crew Socks),(Barcelona Men's Pro Cleats),0.299665,0.166853,0.083287,0.277933,1.665739,1.0,0.033287,1.153837,0.570678,0.217328,0.133326,0.388548
22,(Barcelona Crew Socks),(Barcelona Women's Precision Cleats),0.299665,0.160575,0.083287,0.277933,1.730863,1.0,0.035168,1.162531,0.602931,0.220947,0.139808,0.398306
10,(Barcelona Crew Socks),(Barcelona Men's Eco Cleats),0.299665,0.169224,0.080636,0.269088,1.590123,1.0,0.029926,1.136628,0.529915,0.207690,0.120205,0.372796
14,(Barcelona Crew Socks),(Barcelona Men's Speed Cleats),0.299665,0.157087,0.080497,0.268622,1.710020,1.0,0.033423,1.152500,0.592876,0.213941,0.132321,0.390528
19,(Barcelona Crew Socks),(Barcelona Women's Advanced Cleats),0.299665,0.153460,0.065430,0.218343,1.422800,1.0,0.019443,1.083007,0.424312,0.168766,0.076645,0.322353
25,(Barcelona Crew Socks),(Barcelona Women's Speed Cleats),0.299665,0.115932,0.063895,0.213222,1.839197,1.0,0.029154,1.123656,0.651523,0.181674,0.110048,0.382182
1,(Barcelona Crew Socks),(Barcelona Crew Socks Breathable),0.299665,0.120954,0.063756,0.212756,1.758980,1.0,0.027510,1.116612,0.616118,0.178655,0.104433,0.369931
7,(Barcelona Crew Socks),(Barcelona Elite Socks),0.299665,0.119420,0.063198,0.210894,1.765990,1.0,0.027412,1.115921,0.619340,0.177577,0.103879,0.370050
8,(Barcelona Crew Socks),(Barcelona Keeper Gloves),0.299665,0.118443,0.062221,0.207635,1.753036,1.0,0.026728,1.112564,0.613365,0.174833,0.101175,0.366479


## Save in Json Format

In [65]:
product_categories = dataset[['product', 'product_category']].drop_duplicates().set_index('product').to_dict()['product_category']

In [66]:
product_categories

{"Barcelona Women's Precision Cleats": "Women's Cleats",
 'Barcelona Signature Match Shorts': 'Shorts',
 "Barcelona Women's Advanced Cleats": "Women's Cleats",
 'Barcelona Limited Edition Match Shorts': 'Shorts',
 "Barcelona Women's Classic Cleats": "Women's Cleats",
 'Barcelona Elite Match Shorts': 'Shorts',
 "Barcelona Women's Speed Cleats": "Women's Cleats",
 "Barcelona Men's Elite Cleats": "Men's Cleats",
 'Barcelona Crew Socks Large': 'Crew Socks',
 'Barcelona Utility Shorts': 'Shorts',
 'Barcelona Crew Socks': 'Crew Socks',
 'Barcelona Performance Match Shorts': 'Shorts',
 'Barcelona Elite Socks': 'Crew Socks',
 "Barcelona Men's Speed Cleats": "Men's Cleats",
 'Barcelona Sport Shorts': 'Shorts',
 "Barcelona Men's Eco Cleats": "Men's Cleats",
 'Barcelona Dynamic Socks': 'Crew Socks',
 'Barcelona Retro Match Shorts': 'Shorts',
 'Barcelona Anniversary Match Shorts': 'Shorts',
 'Barcelona Crew Socks Breathable': 'Crew Socks',
 "Barcelona Men's Pro Cleats": "Men's Cleats",
 'Barcelona

In [67]:
recommendations_json={

}

# Extracts the unique antecedents
antecedents = rules_basket['antecedents'].unique()
# Filters the rules where the antecedent matches the current product and sorts them by confidence
for antecedent in antecedents:
    df_rec = rules_basket[rules_basket['antecedents']==antecedent]
    df_rec = df_rec.sort_values('confidence', ascending=False)

    # converts the antecedent into a string key and creates empty list for storing recommendations
    key = "_".join(antecedent)
    recommendations_json[key] = []

    # Loops over the consequents for each rule and ensures the recommendation isn't duplicated
    for _, row in df_rec.iterrows():
        rec_objects = row['consequents']
        for rec_object in rec_objects:
            already_exists = False
            for current_reco_object in recommendations_json[key]:
                if rec_object == current_reco_object['product']:
                    already_exists = True
            if already_exists:
                continue
            # creates a recommendation object with the recommended item, category and confidence
            rec = {
                "product":rec_object,
                "product_category": product_categories[rec_object],
                "confidence": row['confidence']
            }
            
            # Adds recommendation to dictionairy
            recommendations_json[key].append(rec)

In [69]:
import pprint
pprint.pp(recommendations_json)

{'Barcelona Crew Socks Breathable': [{'product': 'Barcelona Crew Socks',
                                      'product_category': 'Crew Socks',
                                      'confidence': 0.5271049596309112}],
 'Barcelona Crew Socks': [{'product': "Barcelona Women's Classic Cleats",
                           'product_category': "Women's Cleats",
                           'confidence': 0.2802607076350093},
                          {'product': "Barcelona Men's Pro Cleats",
                           'product_category': "Men's Cleats",
                           'confidence': 0.2779329608938548},
                          {'product': "Barcelona Women's Precision Cleats",
                           'product_category': "Women's Cleats",
                           'confidence': 0.2779329608938548},
                          {'product': "Barcelona Men's Eco Cleats",
                           'product_category': "Men's Cleats",
                           'confidence': 0.2690875232

In [71]:
import json
with open("api/recommendation_objects/apriori_recommendation.json", "w") as json_file:
    json.dump(recommendations_json, json_file)